--WORKING

**------------------------------------------------------------------------------------------------------------**

In [49]:
from email.message import EmailMessage
import configparser
import datetime
import requests
import smtplib
import csv
import os

In [ ]:
#ping
with open('ping.txt', 'a') as f:
    f.seek(0, 2)
    f.write(f'{datetime.datetime.now()}\n')

In [ ]:
#credentials
config = configparser.ConfigParser()
config.read('config.ini', encoding='utf-8-sig')

# Автоматический скрининг метрик

Данный блокнот предназачен для автоматического отслеживания состояния метрик и отправки отчетов на электронную почту в случае критического изменения

Все ячейки, которые необходимо заполнить самостоятельно в соответствии с необходимой логикой будут помечены **отделяющими линиями**. Те ячейки, которые выполняют остальную "организационную" часть, никак помечаться не будут

Весь представленный ниже код выполняется в соответствии с расписанием. По умолчанию - **каждый час**

**ВАЖНО!** Самая верхняя содержит статус блокнота.
 - при значении **WORKING** ноутбук отрабатывает в соответствии с расписанием
 - при любом другом значении блокнот перестают циклически выполняться

Периодически исполняется только блокнот с названием `auto.ipynb`

Далее будет приведена инструкция по использованию и кастомизации процесса

## 1. Предписания

### Создание пароля

Как было указано выше, в случае критического изменения метрики предполагается отправка уведомления на почту для незамедлительного реагирования. 

Если Вы посчитаете удобным отправлять уведомления от своего лица (т.е с корпоративной почты WB), ниже приведена небольшая инструкция для получения доступа

- 1) Для начала необходимо зайти в настройки Яндекс.ID из-под корпоративного аккаунта
- 2) Далее перейти в раздел **Безопасность**
- 3) В самом низу страницы перейти на **Пароли приложений**
- 4) После чего в разделе **Создать пароль приложения** выберете вариант **Почта (IMAP,POP3,SMTP)**
- 5) Сохраните полученный пароль

Именно этот пароль Вы и будете испольовать для отправки сообщений 

### Файл конфигурации

Для хранения паролей предназначен файл конфигурации. Полученный из пункта выше пароль а также другие авторизационные данные необходимо поместить в вайл `config.ini`

In [ ]:
#      cтруктура файла

#     [credentials]
#     testpassword = testpass
#     ...

Для доступа к данным используйте код `config.get('credentials', 'testpassword')`, где
 - `credentials`- название заголовка (по умолчанию)
 - `testpassword` - поле, содержащее пароль

## 2. Создание метрики

Под метрикой предполагается некоторая численная величина, получаемая в результате выполнения запроса и меняющаяся в зависимости от момента времени, когда запрос был выполнен. 

В ячейке ниже предполагается написание необходимого коннектора к соответствующей базе данных и запрос необходимых метрик

In [ ]:
# Code

Далее словарь `data` ячейке ниже необходимо заполнить следующим образом:
 - в качестве ключа указать названия метрик в качестве строки
 - в качестве значения указать полученные в результате запроса значения метрик

К примеру, если скрипт Python в качестве ответа на SQL запрос записывает в переменную `result` значения метрики DAU, то одна из записией словаря будет выглядеть следующим образом
`data = {"dau": result}`

**------------------------------ОБЯЗАТЕЛЬНО К ЗАПОЛНЕНИЮ---------------------------------------**

In [ ]:
 data = {}

**------------------------------------------------------------------------------------------------------------**

Эти данные будут использованы для записи cvs-файла и последующей обработки

In [ ]:
FILENAME='logs.csv'

In [ ]:
if (data != {}):
    with open(FILENAME, 'a', newline="") as file:
        data["datetime"] = datetime.datetime.now()
        writer = csv.DictWriter(file, fieldnames=list(data.keys()))
        if os.stat(FILENAME).st_size == 0:
            writer.writeheader()
        writer.writerow(data)

## 3. Обнаружение критических значений

Как было указано выше, на основе истории запроса метрик строится и автоматически дополняется cvs файл

Ниже предполагается написание кода для анализа состояния метрик (файла) и создание переменной `is_critical` типа `bool` такой, что
 - если требуется отправка уведомления, переменная должна принимать значение `True`
 - иначе - `False`

**------------------------------ОБЯЗАТЕЛЬНО К ЗАПОЛНЕНИЮ---------------------------------------**

In [18]:
## import pandas as pd
## ...
is_critical: bool = ...

**------------------------------------------------------------------------------------------------------------**

## 4. Отправка уведомлений

In [26]:
def send_notification(username: str,
        password: str, 
        topic: str,
        content: str,
        recipients: list,
        smtp_server = 'smtp.yandex.ru',
        smtp_port = 465):
    
        msg = EmailMessage()
        msg.set_content(content)
        msg['Subject'] = topic
        msg['From'] = username
        msg['To'] = ', '.join(recipients)
        server = smtplib.SMTP_SSL(smtp_server, smtp_port)
        server.login(username, password)
        server.send_message(msg)
        server.quit()

Для отправки уведомления используется функция `send_notification`, в параметры которой передаются:
- `username: str` - Ваша почта в домене @wb.ru
- `password: str` - полученный пароль из п.1
- `topic: str` - тема сообщения
- `content: str` - содержимое сообщения
- `recipients: list[str]` - адресаты. Например `["user1@wb.ru", "user2@wb.ru"]` 

В поле ниже предполагается дописать недостающие параметры

**------------------------------ОБЯЗАТЕЛЬНО К ЗАПОЛНЕНИЮ---------------------------------------**

In [ ]:
if is_critical:
    send_notification(username=...,
                    password=..., 
                    topic=...,
                    content=...,
                    recipients=...)

**------------------------------------------------------------------------------------------------------------**